In [61]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import math
import statistics

In [62]:
gaze_data = pd.read_csv('2022-02-23-11-25.txt', delimiter='\t')
gaze_data = gaze_data.sort_values(by=["gaze_timestamp"], 
                    ascending=[True])



In [63]:
list(gaze_data)

['frameNumber',
 'unityTimeAtLog',
 'cameraPos_x',
 'cameraPos_y',
 'cameraPos_z',
 'cameraMat_R0C0',
 'cameraMat_R0C1',
 'cameraMat_R0C2',
 'cameraMat_R0C3',
 'cameraMat_R1C0',
 'cameraMat_R1C1',
 'cameraMat_R1C2',
 'cameraMat_R1C3',
 'cameraMat_R2C0',
 'cameraMat_R2C1',
 'cameraMat_R2C2',
 'cameraMat_R2C3',
 'cameraMat_R3C0',
 'cameraMat_R3C1',
 'cameraMat_R3C2',
 'cameraMat_R3C3',
 'yTargetPos_x',
 'yTargetPos_y',
 'yTargetPos_z',
 'rTargetPos_x',
 'rTargetPos_y',
 'rTargetPos_z',
 'bTargetPos_x',
 'bTargetPos_y',
 'bTargetPos_z',
 'wTargetPos_x',
 'wTargetPos_y',
 'wTargetPos_z',
 'confidence',
 'gaze_timestamp',
 'unityTimeAtEyeImageCapture',
 'gaze_normal2_x',
 'gaze_normal2_y',
 'gaze_normal2_z',
 'gaze_normal0_x',
 'gaze_normal0_y',
 'gaze_normal0_z',
 'gaze_normal1_x',
 'gaze_normal1_y',
 'gaze_normal1_z',
 'gaze_point_3d_x',
 'gaze_point_3d_y',
 'gaze_point_3d_z',
 'eye_center0_3d_x',
 'eye_center0_3d_y',
 'eye_center0_3d_z',
 'eye_center1_3d_x',
 'eye_center1_3d_y',
 'eye_ce

In [64]:
print(np.array(gaze_data['gaze_timestamp'] - gaze_data['gaze_timestamp'].iloc[0]))

[0.00000000e+00 3.99449992e-03 1.00105000e-02 ... 1.99919595e+01
 1.99959850e+01 1.99999810e+01]


In [65]:
gaze_data['timestamp_sec'] = np.array(gaze_data['gaze_timestamp'] - gaze_data['gaze_timestamp'].iloc[0])
plIndex = np.arange(len(gaze_data['timestamp_sec']))
print(gaze_data['timestamp_sec'])

0        0.000000
1        0.003994
2        0.010010
3        0.014013
4        0.017997
          ...    
4752    19.983989
4753    19.987956
4754    19.991960
4755    19.995985
4756    19.999981
Name: timestamp_sec, Length: 4757, dtype: float64


In [66]:
plIndex[-1]

4756

In [79]:
# Sampling rate for each second
sampling_group = []
for i in plIndex:
    [decimal,unit] = math.modf(gaze_data['timestamp_sec'].loc[i])
    unit = int(unit)
    if len(sampling_group) > unit:
        sampling_group[unit] += 1
    else:
        sampling_group.append(0)
print(sampling_group)

#Checking if sampling rate was consistent
for i in range(len(sampling_group)):
    if(sampling_group[0] != sampling_group[i]):
        print("Sampling rate is inconsistent at", i, "seconds")



[239, 239, 239, 240, 239, 239, 239, 239, 239, 239, 239, 239, 239, 239, 239, 195, 239, 239, 239, 239]
Sampling rate is inconsistent at 3 seconds
Sampling rate is inconsistent at 15 seconds


### Sampling Rate

In [68]:
startTime = gaze_data['gaze_timestamp'].iloc[0]
endTime = gaze_data['gaze_timestamp'].iloc[-1]


print((plIndex[-1]-plIndex[0])/(endTime-startTime))
# print(len(plIndex[startIndex:endIndex])  / (0-100))

237.80022591055064


In [81]:
def get_3d_coordinates(row,coordinates_name):
    return [row['{}_x'.format(coordinates_name)],row['{}_y'.format(coordinates_name)],row['{}_z'.format(coordinates_name)]]

def get_spherical_coordinates(row,coordinates_name):
    coordinates = get_3d_coordinates(row,coordinates_name)
    az = np.rad2deg(np.arctan(np.divide(coordinates[0],coordinates[2])))
    el = np.rad2deg(np.arctan(np.divide(coordinates[1],coordinates[2])))
    return [az,el]

azEl = gaze_data.apply(lambda arbitraryRowName: get_spherical_coordinates(arbitraryRowName,'gaze_point_3d'),axis=1)
gaze_data['gazeInWorldCam_az'],gaze_data['gazeInWorldCam_el'] = zip(*azEl)
print(azEl)

0       [12.800990596173314, -11.750187512144791]
1       [12.797825823305926, -11.703169497282207]
2       [12.651653577633995, -11.638820986299027]
3        [12.51016161448871, -11.875890322134138]
4       [12.574170306125225, -11.814548198232146]
                          ...                    
4752      [2.272671436388002, -7.401334135520808]
4753     [2.2277741962508086, -7.359074139542605]
4754     [2.1305251780841354, -7.501455679179385]
4755     [2.0403622217811317, -7.540114691877899]
4756     [1.8697764360338203, -7.544748155931231]
Length: 4757, dtype: object


In [83]:
gaze_data['vidTimeStamp'] = np.array(gaze_data['gaze_timestamp'] - gaze_data['gaze_timestamp'].iloc[0])
gaze_data['change_in_time'] = gaze_data['vidTimeStamp'].diff()


gaze_data['angular_velocity_az'] = (gaze_data['gazeInWorldCam_az'].diff())
gaze_data['angular_velocity_el'] = (gaze_data['gazeInWorldCam_el'].diff())

gaze_data['angular_velocity'] = round((np.abs(np.sqrt(gaze_data['angular_velocity_az']**2  + gaze_data['angular_velocity_el']**2) / gaze_data['change_in_time'])),2)


angular_velocity_description = gaze_data['angular_velocity'].describe()
print(angular_velocity_description)
threshold = 3*angular_velocity_description['75%']


def IVT_threshold(angular_velocity,threshold):
    idx=np.where(angular_velocity <= threshold)
    
    fixation = np.split(idx[0],np.array(np.where(np.diff(idx) > 1)[1]+1))
    sequence = []
    for f in fixation:
        if len(f) > 1:
            sequence.append((np.min(f), np.max(f)))
    return sequence

fixation_group = (IVT_threshold(gaze_data['angular_velocity'],threshold))
print(len(fixation_group))



for f in fixation_group:
    for p in f:
        fixa = go.Scattergl(
    x=gaze_data['gaze_point_3d_x'],
    y=gaze_data['gaze_point_3d_y'],
    name = 'velInWorldCam_az',
    mode='markers',
    marker_size = 5,
    opacity = 0.8)


fix_fig = dict(data=fixa)

iplot(fix_fig)
        

count    4756.000000
mean       68.640740
std       125.447802
min         0.880000
25%        24.720000
50%        42.415000
75%        66.982500
max      2973.020000
Name: angular_velocity, dtype: float64
92
